In [19]:
def get_lines(filename):
    file = open(filename, 'r+')
    lines = file.readlines()
    #
    lines = map(lambda line : line[:-1],lines)
    file.close()
    return lines

In [20]:
lines = get_lines('./tests/ejercicio1.txt')

In [21]:
import pandas as pd

In [22]:
syntax = pd.DataFrame(data=lines, columns=['Line'])
syntax.head()

,Line
0,Inicio
1,Lea n
2,"Para i=1,n,+1"
3,"Para j=1,n,+1"
4,"Lea A(i,,j)"


In [24]:
syntax["length"] = syntax["Line"].map(lambda line: len(line))
syntax.head()

,Line,length
0,Inicio,6
1,Lea n,5
2,"Para i=1,n,+1",13
3,"Para j=1,n,+1",13
4,"Lea A(i,,j)",11


In [58]:
def get_type(line):
    #If not line.lower().find('x') returns true if x starts at line[0] 
    if not line.lower().find('inicio'):
        return "inicio"
    if not line.lower().find('pare'):
        return "pare"
    if not line.lower().find('para'):
        return "para"
    if not line.lower().find('lea'):
        return "lea"
    if not line.lower().find('esc'):
        return "esc"
    if not line.lower().find('fpara'):
        return "fpara"
    if not line.lower().find('sino'):
        return "sino"
    if not line.lower().find('si'):
        return "si"
    if not line.lower().find('fsi'):
        return "fsi"
    if line.lower().find('='):
        return "assignment"
    return "Indefinite so far"

syntax["Type"] = syntax["Line"].map(get_type)
syntax.head(len(syntax))

,Line,length,Type
0,Inicio,6,inicio
1,Lea n,5,lea
2,"Para i=1,n,+1",13,para
3,"Para j=1,n,+1",13,para
4,"Lea A(i,,j)",11,lea
5,Fpara,5,fpara
6,Fpara,5,fpara
7,"Para i=1,n,+1",13,para
8,"Para j=1,n,+1",13,para
9,"Lea B(i,,j)",11,lea


In [69]:
def process_for(line):
        lower_bound = line[7]
        upper_bound = line[9]
        increment = int(line[11:13])
        return {
            'lower_b' : lower_bound,
            'upper_b' : upper_bound,
            'inc' : increment
        }

In [70]:
def get_statement_runtime(syntax):
    syntax_as_array = syntax.values
    lines_runtimes = []
    lines_data = []
    index = 0
    #get orders for statements
    #after a for the order increases
    #after a end for the order decreases
    #if raises order too
    #each statement has an order 
    for line in syntax_as_array:
        text = line[0]
        line_type = line[2]
        if line_type in ('inicio','pare','fsi','fpara','sino'):
            lines_runtimes.append(0)
            lines_data.append('control_statement')
        elif line_type in ("assignment",'lea','esc'):
            lines_runtimes.append(1) 
            lines_data.append('variable assignment or i/o')
        elif line_type is 'para':
            line_runtime = 'To be determined.'
            line_data = process_for(text)
            lines_runtimes.append(line_runtime)
            lines_data.append(line_data)
        else:
            lines_runtimes.append('Nan')
            lines_data.append({'guess':'para o si'})
        index+=1
    syntax['Runtime'] = lines_runtimes
    syntax['Data'] = lines_data

    




In [71]:
get_statement_runtime(syntax)
syntax.head(len(syntax))

,Line,length,Type,Runtime,Data
0,Inicio,6,inicio,0,control_statement
1,Lea n,5,lea,1,variable assignment or i/o
2,"Para i=1,n,+1",13,para,To be determined,"{'lower_b': '1', 'upper_b': 'n', 'inc': 1}"
3,"Para j=1,n,+1",13,para,To be determined,"{'lower_b': '1', 'upper_b': 'n', 'inc': 1}"
4,"Lea A(i,,j)",11,lea,1,variable assignment or i/o
5,Fpara,5,fpara,0,control_statement
6,Fpara,5,fpara,0,control_statement
7,"Para i=1,n,+1",13,para,To be determined,"{'lower_b': '1', 'upper_b': 'n', 'inc': 1}"
8,"Para j=1,n,+1",13,para,To be determined,"{'lower_b': '1', 'upper_b': 'n', 'inc': 1}"
9,"Lea B(i,,j)",11,lea,1,variable assignment or i/o
